In [1]:
# Uncomment the line below and run this cell to get your data from github into colab (only runnable in colab, not ordinary jupyter notebook):
#! git clone https://github.com/lkriener/music_generation.git && mv music_generation/* . && rm music_generation -r

Cloning into 'music_generation'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (482/482), done.
remote: Compressing objects: 100% (445/445), done.
remote: Total 482 (delta 55), reused 454 (delta 30), pack-reused 0
Receiving objects: 100% (482/482), 390.83 KiB | 530.00 KiB/s, done.
Resolving deltas: 100% (55/55), done.


In [2]:
# Uncomment line to install requirements
#! pip install -r colab_requirements.txt

     |████████████████████████████████| 11.4MB 24.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/ed/f6/fb/5d070524ecf7ba9ed201247a293c01945cfd7f840f8ef338c0
Successfully built pypianoroll


In [55]:
import src.midi_utils as midi_utils
import pygame
from pypianoroll import Multitrack, Track
import pypianoroll
import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Test of pypianoroll library
We will first use the pypianoroll library - https://salu133445.github.io/pypianoroll/ - to convert a midi file into a multitrack object, get the soprano track, transpose it to C major, create a new multitrack object out of it, and finally write it to a new midi file. 

In [56]:
try:
    home_dir
except NameError:
    home_dir = os.getcwd()

os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir

In [57]:
midi_filename = 'bwv110.7.mid'
csv_text = midi_utils.load_to_csv(midi_filename)
# get semitones to C major 
semitones,_ = midi_utils.get_semitones_to_C(csv_text)
print('Semitones to C major : ' +str(semitones))
# get multitrack object from midi 
multitrack = pypianoroll.parse(midi_filename, beat_resolution=4)
# get the soprano track
track_soprano = multitrack.tracks[0]
print("Soprano pianoroll matrix: ")
print(track_soprano.pianoroll)
# transpose the track to C Major 
transposed_track_soprano = pypianoroll.transpose(track_soprano,-semitones)

print("Soprano flattened pianoroll :")
print(midi_utils.flatten_one_hot_pianoroll(track_soprano.pianoroll))
print("Transposed soprano transposed pianoroll :")
print(midi_utils.flatten_one_hot_pianoroll(transposed_track_soprano.pianoroll))

# create a multitrack from the modified track (can be done with a modified pianoroll too)
new_multitrack = Multitrack(tracks=[transposed_track_soprano], beat_resolution=4)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/data/tests_midi/new_multitrack")

Semitones to C major : 2
Soprano pianoroll matrix: 
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Soprano flattened pianoroll :
[ 0  0  0  0 71 71 73 73 74 74 74 74 73 73 73 73 71 71 71  0 71 71 73 73
 74 74 74 74 73 73 73 73 71 71 71 71 78 78 78 78 76 76 76 76 74 74 74 74
 73 73 73  0 73 73 73 73 74 74 74  0 74 74 74 74 76 76 76  0 76 76 76 76
 78 78 78  0 78 78 78 78 76 76 76 76 74 74 74 74 73 73 73 73 73 73 73 73
 71 71 71 71 78 78 78 78 76 76 76 76 74 74 74 74 73 73 73 73 78 78 78 78
 76 76 76 76 74 74 74 74 73 73 73  0 73 73 73 73 74 74 74  0 74 74 74 74
 76 76 76  0 76 76 76 76 78 78 78  0 78 78 78 78 76 76 76 76 74 74 74 74
 73 73 73 73 73 73 73 73 71 71 71 71]
Transposed soprano transposed pianoroll :
[ 0  0  0  0 69 69 71 71 72 72 72 72 71 71 71 71 69 69 69  0 69 69 71 71
 72 72 72 72 71 71 71 71 69 69 69 69 76 76 76 76 74 74 74 74 72 72 72 72
 71 71 71  0 71 71 71 71 72 72 72  0 72 72 72 72 74 74 74  0 

In [58]:
from src.dataset_utils import TrackDataset, get_dataset_representation_from_tracks
from torch.utils.data import DataLoader

from sklearn.preprocessing import StandardScaler

feature_scaler = StandardScaler()
label_scaler = StandardScaler()

# iterate over all midi files of folder
import glob
import os
import numpy as np

try:
    home_dir
except NameError:
    home_dir = os.getcwd()
    

In [59]:
# determine boundaries in all tracks
lower_notes = []
upper_notes = []
list_pianorolls = []
os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir
for midi_file in glob.glob("*.mid"):
    ## load midi file
    csv_text = midi_utils.load_to_csv(midi_file)
    # get semitones to C major 
    semitones,_ = midi_utils.get_semitones_to_C(csv_text)
    # convert to multitrack object
    multitrack = pypianoroll.parse(midi_file, beat_resolution=2)
    # get the soprano track object and transpose it to C major 
    track = pypianoroll.transpose(multitrack.tracks[0], -semitones)
    
    # get pitch range 
    pitch_range = track.get_active_pitch_range()
    lower_notes.append(pitch_range[0])
    upper_notes.append(pitch_range[1])
    
    # get the flattened representation of pianoroll
    pianoroll_flattened = midi_utils.flatten_one_hot_pianoroll(track.pianoroll)
    # add it to the global list of all tracks
    list_pianorolls.append(pianoroll_flattened)

# convert into an array 
all_pianorolls = np.concatenate(list_pianorolls)

In [60]:
# get lower and upper bounds 
global_lower = min(lower_notes)
global_upper = max(upper_notes)
global_diff = global_upper - global_lower
n_notes = global_diff + 2 # we include the silence 
print('Global lower note : '+ str(global_lower))
print('Global upper note : '+ str(global_upper))
print('Number of notes : '+ str(n_notes))

Global lower note : 57
Global upper note : 83
Number of notes : 28


In [61]:
print(all_pianorolls[0:100])

[ 0  0 70  0 70 70 68 68 66 66 65 65 65 65 63 63 63 63 70  0 70 70 72 72
 74 74 75 75 75 75 74 74 74 74 75 75 78 78 77 77 77  0 75 75 75 75 75 75
 75 75 70  0 70 70 68 68 66 66 65 65 65 65 63 63 63 63 70  0 70 70 72 72
 74 74 75 75 75 75 74 74 74 74 75 75 78 78 77 77 77  0 75 75 75 75 75 75
 75 75 70  0]


In [62]:
# scale pianoroll to 0 
all_pianorolls_scaled = midi_utils.scale_pianoroll(all_pianorolls, global_lower)
print(all_pianorolls_scaled[0:100])

[ 0  0 14  0 14 14 12 12 10 10  9  9  9  9  7  7  7  7 14  0 14 14 16 16
 18 18 19 19 19 19 18 18 18 18 19 19 22 22 21 21 21  0 19 19 19 19 19 19
 19 19 14  0 14 14 12 12 10 10  9  9  9  9  7  7  7  7 14  0 14 14 16 16
 18 18 19 19 19 19 18 18 18 18 19 19 22 22 21 21 21  0 19 19 19 19 19 19
 19 19 14  0]


## Define a recurrent-network model 

In [63]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


## Define a LSTM network as for note-to-note melody generation
We now build a network to implement note-to-note melody generation using LSTMs units as well as drop-out of the output. This network is highly inspired from the Tutorial 08 of the class, by replacing characters by notes. 

In [64]:
# Declaring the model
class NoteRNN(nn.Module):
    
    def __init__(self, n_notes, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.n_notes = n_notes 
        #define the LSTM
        self.lstm = nn.LSTM(self.n_notes, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        #define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        #define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, self.n_notes)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        #get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        #pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        #put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

## Training Code

Time for training! We declare a function, where we define an optimizer (Adam) and loss (cross entropy). We then create the training and validation data and initialize the hidden state of the RNN. 
We loop over the training set, each time encoding the data into one-hot vectors, performing forward and backpropagation, and updating the network parameters.

Every once a while, we generate some loss statistics (training loss and validation loss) to let us know if the model is training correctly.

In [65]:
# Declaring the train method
def train(net, data, data2=None, mode="melody_generation", epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: NoteRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    if mode == "harmonization":
        data2, val_data2 = data2[:val_idx], data2[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_notes = net.n_notes 
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        if mode == "melody_generation":
            batch_generator = midi_utils.get_pianoroll_batches(data, batch_size, seq_length)
        elif mode == "harmonization":
            batch_generator = midi_utils.get_pianoroll_batches_harmonization(data, data2, batch_size, seq_length)
            
        for x, y in batch_generator:
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = midi_utils.one_hot_encode_batch(x, n_notes)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.contiguous().view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                
                if mode == "melody_generation":
                    batch_generator_val = midi_utils.get_pianoroll_batches(val_data, batch_size, seq_length)
                elif mode == "harmonization":
                    batch_generator_val = midi_utils.get_pianoroll_batches_harmonization(val_data, val_data2, batch_size, seq_length)
                for x, y in batch_generator_val:
                    # One-hot encode our data and make them Torch tensors
                    x = midi_utils.one_hot_encode_batch(x, n_notes)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.contiguous().view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [66]:
# Define and print the net
n_hidden=256
n_layers=2

net = NoteRNN(n_notes, n_hidden, n_layers)
print(net)

# Declaring the hyperparameters
batch_size = 16
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, all_pianorolls_scaled, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

NoteRNN(
  (lstm): LSTM(28, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=256, out_features=28, bias=True)
)


KeyboardInterrupt: 

In [67]:
""# Defining a method to generate the next character
def predict(net, note, h=None):
        ''' Given a note, predict the next note.
            Returns the predicted note and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[note]])
        x = midi_utils.one_hot_encode_batch(x, net.n_notes)
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        note_range = np.arange(net.n_notes)
        # select the likely next note with some element of randomness
        p = p.numpy().squeeze()
        note = np.random.choice(note_range, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return note, h

In [83]:
# Declaring a method to generate new melody
def sample(net, size, prime=[10,10,12,12]):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    notes = [no for no in prime]
    h = net.init_hidden(1)
    for no in prime:
        note, h = predict(net, no, h)
    notes.append(note)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        note, h = predict(net, notes[-1], h)
        notes.append(note)

    return np.array(notes)


# Declaring a method to generate new melody
def sample_harmonization(net, seq):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    size = len(seq)
    notes = []
    h = net.init_hidden(1)
    # Now pass in the previous character and get a new one
    for ii in range(size):
        note, h = predict(net, seq[ii], h)
        notes.append(note)

    return np.array(notes)




In [84]:
# Generating new melody
notes = sample(net, 100, prime=[1,1,1,1])
print("Generated sample :")
print(notes)
normal_pianoroll = midi_utils.unscale_pianoroll(notes, global_lower) # go back to the track range
print("Generated sample in the pitch range :")
print(normal_pianoroll)
# create a one_hot_pianoroll
one_hot_pianoroll = midi_utils.one_hot_encode_pianoroll(normal_pianoroll, 128)*90 # 90 for the velocity
# store it a in a track object
new_track = Track(pianoroll=one_hot_pianoroll, name='new track')
# create a multitrack made of the generated track object
new_multitrack = Multitrack(tracks=[new_track], tempo = 90, beat_resolution=2)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/results/RNN_track")


Generated sample :
[ 1  1  1  1  3  6  9 13 12 15 26 20 18  8 21 22 15 17 14 13 21 11 24  0
 19 16 12 14 10 19 11 23 16 15  8  6 13  8 10 17 18 18 10 22 16 16 14 15
 10  0 15  8  9 17 14 15 15  9 21 16 18 15 14 13  9  0  0 21 14 10 17 13
 14 14 15 10 17 17 13  9 12 10 15  8 12 22 13  7  8 14 13 15 13 17 21 12
  4  0 11 14 13 14 10 18 15]
Generated sample in the pitch range :
[51 51 51 51 53 56 59 63 62 65 76 70 68 58 71 72 65 67 64 63 71 61 74  0
 69 66 62 64 60 69 61 73 66 65 58 56 63 58 60 67 68 68 60 72 66 66 64 65
 60  0 65 58 59 67 64 65 65 59 71 66 68 65 64 63 59  0  0 71 64 60 67 63
 64 64 65 60 67 67 63 59 62 60 65 58 62 72 63 57 58 64 63 65 63 67 71 62
 54  0 61 64 63 64 60 68 65]


In [85]:
modified_midi_filename = home_dir + "/results/RNN_track.mid"
pygame.init()
pygame.mixer.music.load(modified_midi_filename)
pygame.mixer.music.play()

In [86]:
pygame.mixer.music.stop()

## Train for harmonization

In [87]:
# determine boundaries in all tracks
lower_notes = []
upper_notes = []
list_pianorolls = []
list_pianorolls2 = []
os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir
for midi_file in glob.glob("*.mid"):
    ## load midi file
    csv_text = midi_utils.load_to_csv(midi_file)
    # get semitones to C major 
    semitones,_ = midi_utils.get_semitones_to_C(csv_text)
    # convert to multitrack object
    multitrack = pypianoroll.parse(midi_file, beat_resolution=4)
    # get the soprano track object and transpose it to C major 
    track = pypianoroll.transpose(multitrack.tracks[0], -semitones)
    track2 = pypianoroll.transpose(multitrack.tracks[1], -semitones)
    # get pitch range 
    pitch_range = track.get_active_pitch_range()
    lower_notes.append(pitch_range[0])
    upper_notes.append(pitch_range[1])
    pitch_range2 = track2.get_active_pitch_range()
    lower_notes.append(pitch_range2[0])
    upper_notes.append(pitch_range2[1])
    
    
    
    # get the flattened representation of pianoroll
    pianoroll_flattened = midi_utils.flatten_one_hot_pianoroll(track.pianoroll)
    pianoroll_flattened2 = midi_utils.flatten_one_hot_pianoroll(track2.pianoroll)
    # add it to the global list of all tracks
    list_pianorolls.append(pianoroll_flattened)
    list_pianorolls2.append(pianoroll_flattened2)


In [88]:
# convert into an array 
all_pianorolls = np.concatenate(list_pianorolls)
all_pianorolls2 = np.concatenate(list_pianorolls2)
# get lower and upper bounds 
global_lower = min(lower_notes)
global_upper = max(upper_notes)
global_diff = global_upper - global_lower
n_notes = global_diff + 2 # we include the silence 
print('Global lower note : '+ str(global_lower))
print('Global upper note : '+ str(global_upper))
print('Number of notes : '+ str(n_notes))

Global lower note : 51
Global upper note : 83
Number of notes : 34


In [89]:
print("Voice to train on :")
print(all_pianorolls[0:100])
print("Voice to target :")
print(all_pianorolls2[0:100])

Voice to train on :
[ 0  0  0  0 70 70 70  0 70 70 70 70 68 68 68 68 66 66 66 66 65 65 65 65
 65 65 65 65 63 63 63 63 63 63 63 63 70 70 70  0 70 70 70 70 72 72 72 72
 74 74 74 74 75 75 75 75 75 75 75 75 74 74 74 74 74 74 74 74 75 75 75 75
 78 78 78 78 77 77 77 77 77 77 75  0 75 75 75 75 75 75 75 75 75 75 75 75
 75 75 75 75]
Voice to target :
[ 0  0  0  0 66 66 66  0 66 66 65 65 63 63 63  0 63 63 63  0 63 63 63 63
 62 62 62 62 58 58 58 58 58 58 58 58 66 66 66  0 66 66 66 66 68 68 66 66
 65 65 65 65 63 63 63 63 65 65 65  0 65 65 65 65 65 65 65 65 66 66 68 68
 70 70 70  0 70 70 70 70 68 68 68 68 66 66 66 66 66 66 66 66 66 66 66 66
 66 66 66  0]


In [90]:
# scale pianoroll to 0 
all_pianorolls_scaled = midi_utils.scale_pianoroll(all_pianorolls, global_lower)
print("Scaled voice to train on :")
print(all_pianorolls_scaled[0:100])
all_pianorolls_scaled2 = midi_utils.scale_pianoroll(all_pianorolls2, global_lower)
print("Scaled voice to target :")
print(all_pianorolls_scaled2[0:100])

Scaled voice to train on :
[ 0  0  0  0 20 20 20  0 20 20 20 20 18 18 18 18 16 16 16 16 15 15 15 15
 15 15 15 15 13 13 13 13 13 13 13 13 20 20 20  0 20 20 20 20 22 22 22 22
 24 24 24 24 25 25 25 25 25 25 25 25 24 24 24 24 24 24 24 24 25 25 25 25
 28 28 28 28 27 27 27 27 27 27 25  0 25 25 25 25 25 25 25 25 25 25 25 25
 25 25 25 25]
Scaled voice to target :
[ 0  0  0  0 16 16 16  0 16 16 15 15 13 13 13  0 13 13 13  0 13 13 13 13
 12 12 12 12  8  8  8  8  8  8  8  8 16 16 16  0 16 16 16 16 18 18 16 16
 15 15 15 15 13 13 13 13 15 15 15  0 15 15 15 15 15 15 15 15 16 16 18 18
 20 20 20  0 20 20 20 20 18 18 18 18 16 16 16 16 16 16 16 16 16 16 16 16
 16 16 16  0]


In [91]:
# Define and print the net
n_hidden=256
n_layers=2

net_harmonization = NoteRNN(n_notes, n_hidden, n_layers)
print(net)

# Declaring the hyperparameters
batch_size = 16
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net_harmonization, all_pianorolls_scaled, all_pianorolls_scaled2, mode="harmonization", epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

NoteRNN(
  (lstm): LSTM(28, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=256, out_features=28, bias=True)
)


KeyboardInterrupt: 

In [95]:
# Generating harmonization

# get one voice from the dataset 
midi_filename = 'bwv110.7.mid'
csv_text = midi_utils.load_to_csv(midi_filename)
# get semitones to C major 
semitones,_ = midi_utils.get_semitones_to_C(csv_text)
print('Semitones to C major : ' +str(semitones))
# get multitrack object from midi 
multitrack = pypianoroll.parse(midi_filename, beat_resolution=4)
# get the soprano track
soprano_track = multitrack.tracks[0]
# transpose the track to C Major 
soprano_track = pypianoroll.transpose(soprano_track,-semitones)
soprano_pianoroll = midi_utils.flatten_one_hot_pianoroll(transposed_track_soprano.pianoroll)
print('Soprano pianoroll :')
print(soprano_pianoroll)
scaled_soprano_pianoroll = midi_utils.scale_pianoroll(soprano_pianoroll, global_lower)
# now we can predict the second voice 
scaled_alto_pianoroll = sample_harmonization(net, scaled_soprano_pianoroll)

alto_pianoroll = midi_utils.unscale_pianoroll(notes, global_lower)
print('Alto pianoroll :')
print(alto_pianoroll)
one_hot_alto = midi_utils.one_hot_encode_pianoroll(alto_pianoroll, 128)*90
alto_track = Track(pianoroll=one_hot_alto, name='alto gernerated track')
# create a multitrack made of the initial soprano track and the generated alto
new_multitrack = Multitrack(tracks=[soprano_track, alto_track], tempo = 90, beat_resolution=4)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/results/RNN_harmonization_track")

Semitones to C major : 2
Soprano pianoroll :
[ 0  0  0  0 69 69 71 71 72 72 72 72 71 71 71 71 69 69 69  0 69 69 71 71
 72 72 72 72 71 71 71 71 69 69 69 69 76 76 76 76 74 74 74 74 72 72 72 72
 71 71 71  0 71 71 71 71 72 72 72  0 72 72 72 72 74 74 74  0 74 74 74 74
 76 76 76  0 76 76 76 76 74 74 74 74 72 72 72 72 71 71 71 71 71 71 71 71
 69 69 69 69 76 76 76 76 74 74 74 74 72 72 72 72 71 71 71 71 76 76 76 76
 74 74 74 74 72 72 72 72 71 71 71  0 71 71 71 71 72 72 72  0 72 72 72 72
 74 74 74  0 74 74 74 74 76 76 76  0 76 76 76 76 74 74 74 74 72 72 72 72
 71 71 71 71 71 71 71 71 69 69 69 69]
Alto pianoroll :
[51 51 51 51 53 56 59 63 62 65 76 70 68 58 71 72 65 67 64 63 71 61 74  0
 69 66 62 64 60 69 61 73 66 65 58 56 63 58 60 67 68 68 60 72 66 66 64 65
 60  0 65 58 59 67 64 65 65 59 71 66 68 65 64 63 59  0  0 71 64 60 67 63
 64 64 65 60 67 67 63 59 62 60 65 58 62 72 63 57 58 64 63 65 63 67 71 62
 54  0 61 64 63 64 60 68 65]


In [98]:
modified_midi_filename = home_dir + "/results/RNN_harmonization_track.mid"
pygame.init()
pygame.mixer.music.load(modified_midi_filename)
pygame.mixer.music.play()

In [99]:
pygame.mixer.music.stop()